In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import json
import re

In [10]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("서대문구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수 설정

while page_number <= MAX_PAGES:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:  # 마지막 페이지에서는 클릭하지 않음
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# JSON 파일 저장
file_name = "seodaemun_restaurants.json"
with open(file_name, "w", encoding="utf-8") as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 현명식탁B1
2. 연희보리밥
3. 이석덕생면파스타 신촌점
4. 한사발포차 신촌점
5. 머노까머나 신촌점
6. 스웨이커피스테이션
7. 정육면체 신촌점
8. 작은스페인
9. 아민 이화
10. 목란
11. 연경 연희점
12. 이빠네마 그릴
13. 시오 연희동본점
14. 몽주방 신촌점
15. 만동제과 연남점
16. 한술식당 신촌점
17. 설레임삼겹살 신촌점
18. 하남돼지집 서대문역점
19. 다대기실비 서대문역
20. 아건 이대본점
21. 파이홀
22. 한옥집김치찜 본점
23. 엠브로돈까스 신촌본점
24. 오마카세 오사이초밥 신촌점
25. 치즈웨이브
26. 화로상회 신촌점
27. 라구식당
28. 강남불백 신촌점
29. 마우디
30. 백순대본가새맛 신촌연대점
31. 담솥 신촌점
32. 쟁반집8292 신촌점
33. 수빈
34. 산천마루
35. 쿳사 연희
36. 칠백식당
37. 마이시크릿메이트
38. 청안식탁
39. 센트그릴
40. 담산 신촌본점
41. 월순철판동태찜
42. 한술식당 명지대점
43. 비아37 신촌본점
44. 녹원쌈밥 연희점
45. 수지상회 신촌점
46. 신촌고기창고
47. 키친봄날
48. 평안도식당 신촌점
49. 포가레 신촌점
50. 고래고래신촌점
51. 혼신꼬치 신촌점
52. 돈불1971
53. 끝집 가재울본점
54. 유우 연희동본점
55. 다방방
56. 카라멘야
57. 타코몽
58. 가마마루이라멘
59. 당나발포차 신촌점
60. 가야가야
61. 복성각
62. 마더린러 베이글
63. 오르랔베이커리
64. 피터팬1978
65. 푸어링아웃
66. 봉일천 장군집
67. 다이닝후
68. 카츠업
69. 시트플레이스
70. 오향만두
71. 포티드
72. 처음본명품민물장어 홍제점
73. 고삼이 신촌점
74. 티앙팡 오후의 홍차
'다음' 버튼 클릭: 페이지 2로 이동 중
현재 2페이지) 총 70개의 가게를 찾았습니다.
75. 교촌치킨 명지대점
76. 닭발상점 명지대점
77. 대빈
78. 미분당 가좌점
79. 프랭크버

가게 크롤링 중 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00EB3433+25059]
	(No symbol) [0x00E3CE34]
	(No symbol) [0x00D1BEC3]
	(No symbol) [0x00CFD93B]
	(No symbol) [0x00D8800F]
	(No symbol) [0x00D9AE49]
	(No symbol) [0x00D81C96]
	(No symbol) [0x00D53FAC]
	(No symbol) [0x00D54F3D]
	GetHandleVerifier [0x011A5593+3113795]
	GetHandleVerifier [0x011BA25A+3198986]
	GetHandleVerifier [0x011B2A32+3168226]
	GetHandleVerifier [0x00F532A0+680016]
	(No symbol) [0x00E4577D]
	(No symbol) [0x00E42A28]
	(No symbol) [0x00E42BC5]
	(No symbol) [0x00E35820]
	BaseThreadInitThunk [0x763BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x770F809E+286]
	RtlGetAppContainerNamedObjectPath [0x770F806E+238]

가게 크롤링 중 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00EB343